In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from datetime import datetime

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 16
NUM_WORKERS = 2
NORMALIZE_MEAN = [0.485, 0.456, 0.406]
NORMALIZE_STD = [0.229, 0.224, 0.225]
NUM_CLASSES = 9
NUM_EPOCHS = 30

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToNormalize(NORMALIZE_MEAN, NORMALIZE_STD),
    transforms.ToTensor(),
])

val_transforms = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE * 1.14)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
    transforms.ToTensor(),
])

train_dataset_path = "../../../dataset/train"
val_dataset_path = "../../../dataset/val"
train_dataset = datasets.ImageFolder(root=train_dataset_path, transform=train_transforms)
val_dataset = datasets.ImageFolder(root=val_dataset_path, transform=val_transforms)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=True,
    pin_memory=True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=False,
    pin_memory=True,
)

In [1]:
if __name__ == "__main__":
    images, labels = next(iter(train_loader))
    print(f"Batch shape: {images.shape}")
    print(f"Label shape: {labels.shape}")

model = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)


for param in model.features.parameters():
    param.requires_grad = False

old_linear = model.classifier[3]
in_feats = old_linear.in_features

classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(in_feats, NUM_CLASSES),
)
model.classifier[3] = classifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters())

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += images.size(0)

    return val_loss/total, correct/total

def validate(model, loader, criterion, device):
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += images.size(0)

    return val_loss/total, correct/total

NameError: name 'train_loader' is not defined

In [ ]:
best_val_loss = float('inf')

for epoch in range (NUM_EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}: "
          f"Train loss {train_loss:.4f}, acc {train_acc:.4f} | "
          f"Val loss {val_loss:.4f}, acc {val_acc:.4f}")

    # checkpoint best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        now = datetime.now()
        model_name = "./mobilenet/v3/" + "mobilenetv3" + "-" + now.strftime("%m%d%Y%H%M%S") 
        torch.save(model.state_dict(), model_name)